<a href="https://colab.research.google.com/github/jubaal91/GAAT/blob/main/Introduccion_PR_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción a la Percepción Remota con GEE




### Librerías y Pres

In [1]:
#Librerías comunes para trabajar con imagenes
import pandas as pd
import geopandas as gpd

#Earth Engine: Nos permite visualizar y manipular imagenes de satélite sin necesidad de descargarlas
import ee

#Permite visualizar imagenes dentro del notebook
from IPython.display import Image

#Leer las fechas
import datetime

In [2]:
#Autentificación para acceder a EE
ee.Authenticate()

# Inicialización, es necesario definir el proyecto donde se trabajará
ee.Initialize(project='ee-jubaal91')

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


In [17]:
#Coordenadas del Incendio

lat = 22.07683
lon = -101.00216

#Punto de interés tomando las coordenadas
poi = ee.Geometry.Point(lon,lat)

#Fecha de inicio
start_date = '2024-04-01'

#Fecha final
end_date = '2024-04-30'

In [18]:
#Obtener las imagenes del satélite Sentinel 2
sentinel = ee.ImageCollection("COPERNICUS/S2_SR")\
            .filterBounds(poi)\
            .filterDate(start_date,end_date)

In [19]:
#¿Cuántas imagenes hay de la zona?
print('Total number:', sentinel.size().getInfo())

Total number: 7


In [20]:
#Información acerca de la primera imagen
#Es util para conocer el nombre de los campos, en caso de que sea necesario consultar información de ellos
sentinel.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [1830, 1830],
   'crs': 'EPSG:32614',
   'crs_transform': [60, 0, 199980, 0, -60, 2500020]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32614',
   'crs_transform': [10, 0, 199980, 0, -10, 2500020]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32614',
   'crs_transform': [10, 0, 199980, 0, -10, 2500020]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32614',
   'crs_transform': [10, 0, 199980, 0, -10, 2500020]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min':

In [21]:
#¿Cual es el porcentaje de nubes que cubren a la primera imagen?
sentinel.first().get('CLOUDY_PIXEL_PERCENTAGE').getInfo()

0.005912

In [22]:
#¿Cuando fue tomada esta imagen?
datetime.datetime.fromtimestamp(sentinel.first().get('system:time_start').getInfo()/1000).strftime('%Y-%m-%d %H:%M:%S')


'2024-04-03 17:27:45'

In [23]:
#'Cuántas bandas tenemos?
sentinel.first().bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B8A',
 'B9',
 'B11',
 'B12',
 'AOT',
 'WVP',
 'SCL',
 'TCI_R',
 'TCI_G',
 'TCI_B',
 'MSK_CLDPRB',
 'MSK_SNWPRB',
 'MSK_CLASSI_OPAQUE',
 'MSK_CLASSI_CIRRUS',
 'MSK_CLASSI_SNOW_ICE']

In [24]:
#Guarda todas las imagenes en una lista
sentinel_list = sentinel.toList(sentinel.size());

In [25]:
#Configura algunos parametros para las imagenes
parameters = {
                'min': 0,
                'max': 3000,
                'dimensions': 800, # square size in pixels
                'bands': ['B4', 'B3', 'B2'] # bands to display (r,g,b)
             }

In [26]:
import datetime

#Crea un contenedor de datos vacio
data = []

#Ciclo que recorre cada una de las imagenes y las despliega
for i in range(sentinel.size().getInfo()):

    #Fecha de captura de las imagenes
    timestamp_ms = ee.Image(sentinel_list.get(i)).get('system:time_start').getInfo()
    timestamp_s = timestamp_ms / 1000
    date_gregorian = datetime.datetime.fromtimestamp(timestamp_s).strftime('%Y-%m-%d %H:%M:%S')


    #Porcentaje de nubosidad
    cloud = ee.Image(sentinel_list.get(i)).get('CLOUDY_PIXEL_PERCENTAGE').getInfo()

    #Muestra la información de la imagen
    print('Image #',i,date_gregorian,'Cloud cover:',cloud)

    #Despliega la imagen
    display(Image(url = ee.Image(sentinel_list.get(i)).getThumbUrl(parameters)))

    #Convierte esta información (i, fecha, nubosidad) en una lista
    this_data = [i,date_gregorian,cloud]

    #Va agregando información a la lista
    data.append(this_data)


#Crea un data frame
df = pd.DataFrame(data, columns = ['Image #', 'Date', 'Cloud Cover'])

Image # 0 2024-04-03 17:27:45 Cloud cover: 0.005912


Image # 1 2024-04-08 17:27:47 Cloud cover: 0


Image # 2 2024-04-13 17:27:50 Cloud cover: 58.581382


Image # 3 2024-04-18 17:27:49 Cloud cover: 2.205424


Image # 4 2024-04-23 17:27:50 Cloud cover: 0.001437


Image # 5 2024-04-28 17:27:43 Cloud cover: 0.587107


Image # 6 2024-04-28 17:27:50 Cloud cover: 0.449671


In [27]:
#Datos del data frame
df

,Image #,Date,Cloud Cover
0,0,2024-04-03 17:27:45,0.005912
1,1,2024-04-08 17:27:47,0.000000
2,2,2024-04-13 17:27:50,58.581382
3,3,2024-04-18 17:27:49,2.205424
4,4,2024-04-23 17:27:50,0.001437
5,5,2024-04-28 17:27:43,0.587107
6,6,2024-04-28 17:27:50,0.449671


In [37]:
# create a list of images we want (before, during, after)
sentinel_sequence = [0,1,4]

In [34]:
# Define a region of interest with a buffer zone of 20 km
roi = poi.buffer(5000) # meters

In [35]:
parameters = {
                'min': 0,
                'max': 3000,
                'dimensions': 800,
                'bands': ['B4', 'B3', 'B2'],
                'region':roi
             }

In [38]:
for i in sentinel_sequence:

    # when was this image taken?
    date = ee.Image(sentinel_list.get(i)).get('DATE_ACQUIRED').getInfo()

    # cloud cover
    cloud = ee.Image(sentinel_list.get(i)).get('CLOUD_COVER').getInfo()

    print('Image #',i,date,'Cloud cover:',cloud)

    display(Image(url = ee.Image(sentinel_list.get(i)).getThumbUrl(parameters)))

Image # 0 None Cloud cover: None


Image # 1 None Cloud cover: None


Image # 4 None Cloud cover: None


###Indices de Vegetación

In [44]:
# ndvi palette: red is low, green is high vegetation
palette = ['red', 'yellow', 'green']

ndvi_parameters = {'min': 0,
                   'max': 0.4,
                   'dimensions': 512,
                   'palette': palette,
                   'region': roi}

In [45]:
for i in sentinel_sequence:

    # when was this image taken?
    date = ee.Image(sentinel_list.get(i)).get('DATE_ACQUIRED').getInfo()

    # print some information
    print('Image #',i,date)

    # display the image
    display(Image(url=ee.Image(sentinel_list.get(i)).normalizedDifference(['B8', 'B4']).getThumbUrl(ndvi_parameters)))

Image # 0 None


Image # 1 None


Image # 4 None
